### Unlabelled Tranfer Learning (Contextual Approach)

Pretrained base model with weight vocabulary from en_core_web_lg(large) dataset

In [1]:
import os
import spacy
import tempfile
import en_core_web_lg

In [3]:
base = 'en_core_web_lg' # Also try with vectors
nlp = en_core_web_lg.load()

In [4]:
# name = os.path.join(tempfile.gettempdir(), 'nlp/test_data_01')
# nlp = spacy.load(name)

In [5]:
text = 'Uber blew through $1 million a week'
doc = nlp(text)
ents = set([(ent.text, ent.label_) for ent in doc.ents])

In [6]:
ents

{('$1 million', 'MONEY')}

### Base model + contextual vocabulary of target entity.

Save base model(base model with weights obtained from training over <b>en_core_web_lg</b>)

In [2]:
from pathlib import Path
import os

In [28]:
output_dir = "model_saves"
output_dir = Path(os.path.join(Path().absolute(), Path(output_dir)))
if not output_dir.exists():
    output_dir.mkdir()
nlp.to_disk(output_dir)
print("Saved model to", output_dir)

Saved model to /home/pratik/Myfiles/Notebooks/Autotagging/test_scripts/model_saves


In [29]:
!ls

base_model_contextual_TF.ipynb	entity_ruler.ipynb  pretrained_test.ipynb
base_model_pred.ipynb		model		    spacy_regex_test.ipynb
basic_ner.ipynb			model_saves


In [ ]:
#!python -m spacy pretrain ub.jsonl model_saves w2s

Test model prediction capability

In [3]:
from __future__ import unicode_literals, print_function, annotations
import plac
from spacy.util import minibatch, compounding

In [4]:
@plac.annotations(
    model=("Model name. Defaults to blank 'en' model.", "option", "m", str),
    output_dir=("Optional output directory", "option", "o", Path),
    init_tok2vec=("Pretrained tok2vec weights", "option", "t2v", Path)
)
def test_bwc(model=None, output_dir=None, init_tok2vec=None):
    """Load the model, set up the pipeline and train the entity recognizer."""
    if model is not None:
        nlp = spacy.load(model)  # load pretrained bwc model
        print("Loaded model '%s'" % model)
    else:
        nlp = spacy.blank("en")  # create blank Language class
        print("Created blank 'en' model")

    # create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if "ner" not in nlp.pipe_names:
        ner = nlp.create_pipe("ner")
        nlp.add_pipe(ner, last=True)
    # otherwise, get it so we can add labels
    else:
        ner = nlp.get_pipe("ner")

    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
    with nlp.disable_pipes(*other_pipes):  # only train NER
        init_tok2vec=Path(init_tok2vec)
        if init_tok2vec is not None:
            with init_tok2vec.open("rb") as file_:
                ner.model.tok2vec.from_bytes(file_.read())
                
        # reset and initialize the weights randomly – but only if we're
        # training a new model
#         if model is None:
#             nlp.begin_training()
#         for itn in range(n_iter):
#             random.shuffle(TRAIN_DATA)
#             losses = {}
#             # batch up the examples using spaCy's minibatch
#             batches = minibatch(TRAIN_DATA, size=compounding(4.0, 32.0, 1.001))
#             for batch in batches:
#                 texts, annotations = zip(*batch)
#                 nlp.update(
#                     texts,  # batch of texts
#                     annotations,  # batch of annotations
#                     drop=0.5,  # dropout - make it harder to memorise data
#                     losses=losses,
#                 )
#             print("Losses", losses)

#     # test the trained model
#     for text, _ in TRAIN_DATA:
#         doc = nlp(text)
#         print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
#         print("Tokens", [(t.text, t.ent_type_, t.ent_iob) for t in doc])

    # save model to output directory
    if output_dir is not None:
        output_dir = Path(output_dir)
        if not output_dir.exists():
            output_dir.mkdir()
        nlp.to_disk(output_dir)
        print("Saved model to", output_dir)

#         # test the saved model
#         print("Loading from", output_dir)
#         nlp2 = spacy.load(output_dir)
#         for text, _ in TRAIN_DATA:
#             doc = nlp2(text)
#             print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
#             print("Tokens", [(t.text, t.ent_type_, t.ent_iob) for t in doc])

In [5]:
test_bwc(model="model_saves", output_dir="f2s", init_tok2vec="w2s/model0.bin")

Loaded model 'model_saves'


ValueError: could not broadcast input array from shape (96,3,384) into shape (96,3,480)